In [1]:
# Check python version and working dir
! python -V
! ls .

Python 2.7.17
 __init__.py	       upload_from_csv.py	 vericast_reports.py
'Programs API.ipynb'   uploading_epg_csv.ipynb	 Vericast-reports-tool.ipynb


In [2]:
import os
import json
import pandas as pd
import datetime
import dateutil.parser
         

# Change directory to filesdir
os.chdir('../../data/')
class VericastMatchReporter:
    """
    @params: filename, start_date, start_time, end_date, end_time, time_zone
    """
    filename = None
    channel = None
    report_name = 'vericast-api-matches'
    report_file_extension = 'xslx'
    df = pd.DataFrame(columns=['title','length','album','artist','start_time_utc'])
    matches_between_dates = None
    start_time = None
    end_time = None
    dfeng = None
    
    
    def __init__(self, **kwargs):
        """Initilize report, make a pandas dataframe with the matches."""
        
        self.filename = kwargs['filename']
        self.channel = self.filename.split('/')[-1].split('-')[-1].split('.')[0]
        self.report_name += '-' + self.channel 
        try:
            # Make a df with engineer's times and bring them to UTC for correct comparison
            start_time_naive = dateutil.parser.parse(kwargs['start_date']+'T'+kwargs['start_time'])
            end_time_naive = dateutil.parser.parse(kwargs['end_date']+'T'+kwargs['end_time'])
            dfeng = pd.DataFrame({'naive_datetime':[start_time_naive,end_time_naive]}, index=['filter_start','filter_end'])
            dfeng['local_datetime'] = pd.DatetimeIndex(dfeng['naive_datetime']).tz_localize(tz =kwargs['time_zone'])
            dfeng['utc_datetime'] = pd.DatetimeIndex(dfeng['local_datetime']).tz_convert(tz ='UTC')
            self.dfeng = dfeng
            self.report_name +=' {} to {}.'.format(self.dfeng.loc['filter_start']['local_datetime'],self.dfeng.loc['filter_end']['local_datetime'])
            self.report_name =os.getcwd() + '.{}'.format(self.report_file_extension)
        except:
            raise
        with open(self.filename) as f:
            for line in f.readlines():
                # Append line into df
                self.df = self.df.append(json.loads(line), ignore_index=True)
                
        # Cast dates as aware datetime with timezone 'UTC'
        self.df['start_time_utc'] = pd.to_datetime(self.df['start_time_utc'], yearfirst=True, utc=True)
        print('Reporter initialized.')
        
    def create_report_between_times_xlsx(self):
        # Make a mask boolean mask between dates
        mask = (self.df['start_time_utc'] >= self.dfeng.loc['filter_start']['utc_datetime']) & (self.df['start_time_utc'] <= self.dfeng.loc['filter_end']['utc_datetime'])
        self.matches_between_dates = self.df.loc[mask]
        # Write report to xslx
        writer = pd.ExcelWriter(self.report_name)
        matches_between_dates.to_excel(writer, sheet_name=self.channel, index=False)
        writer.save()
        writer.close()
        print('File {} with {} matches has been created.'.format(self.report_name, self.matches_between_dates.count()))
        return self.matches_between_dates
    
    def __str__(self):
        return 'Vericast API matches from '+self.channel.replace('_', ' ').upper()+' between times '+str(self.start_time)+' and '+str(self.end_time)+' UTC.'

    def __repr__(self):
        return "<{}: Channel {}>".format(self.__class__.__name__, self.channel)

In [3]:
# Create a reporter instance with engineer's data
report = VericastMatchReporter(
    filename='matches-la_sexta.json', 
    start_date='2018-01-29', 
    start_time='12:00', 
    end_date='2018-02-04',
    end_time='20:13',
    time_zone='america/new_york'
)

Reporter initialized.


In [5]:
# show the df created by reporter from file
sample = report.df.sample(100).sort_values(by=['start_time_utc'])
sample

,title,length,album,artist,start_time_utc
470,Eye Spy,17,Vol. 8: Spy Themes Remixed,"Chuck Cassey, Jules Bromley",2018-01-25 17:01:05+00:00
469,I Like You (instrumental),14,"Offbeat, Cute & Quirky - Instrumentals",Phil Buckle,2018-01-25 18:12:01+00:00
465,Inception,3,Hybrid Electro,Boris Nonte,2018-01-25 21:43:10+00:00
459,Eye Spy,17,Vol. 8: Spy Themes Remixed,"Chuck Cassey, Jules Bromley",2018-01-26 00:44:08+00:00
458,Inception,3,Hybrid Electro,Boris Nonte,2018-01-26 00:48:01+00:00
452,Those Three Words,15,Abstract Future - Let You In,"Andrew Haas, Ian Franzino, Laura Lane",2018-01-26 12:07:01+00:00
445,Rainbow Odyssey,2,Electro Spectrum,"Josselin Bordat, Le Fat Club",2018-01-26 18:09:09+00:00
444,Eye Spy,17,Vol. 8: Spy Themes Remixed,"Chuck Cassey, Jules Bromley",2018-01-26 22:50:14+00:00
438,Night Surveillance,66,Night Watch,Frederic Sans,2018-01-27 00:26:44+00:00
437,Eye Spy,15,Vol. 8: Spy Themes Remixed,"Chuck Cassey, Jules Bromley",2018-01-27 00:40:22+00:00


In [6]:
# Make a mask boolean mask between dates
mask = (sample['start_time_utc'] >= report.dfeng.loc['filter_start']['utc_datetime']) & (sample['start_time_utc'] <= report.dfeng.loc['filter_end']['utc_datetime'])
matches_between_dates = sample.loc[mask]

In [7]:
dirw= os.getcwd()
writer = pd.ExcelWriter(dirw+'my_file32.xlsx')
matches_between_dates.to_excel(writer, sheet_name='Whatever', index=False)
writer.save()
writer.close()

In [8]:
# show the dfeng created by reporter from engineer's data
report.dfeng.head(2)

,naive_datetime,local_datetime,utc_datetime
filter_start,2018-01-29 12:00:00,2018-01-29 12:00:00-05:00,2018-01-29 17:00:00+00:00
filter_end,2018-02-04 20:13:00,2018-02-04 20:13:00-05:00,2018-02-05 01:13:00+00:00


In [9]:
matches_between_dates

,title,length,album,artist,start_time_utc
362,Simple And Pure,9,Sunny And Simple,"John Albert Griggs, Philip Michael Guyler",2018-01-29 23:43:51+00:00
360,Never Get Enough,20,Stacks of Soul Wax,"Marc Ferrari, Michael McGregor",2018-01-30 11:11:14+00:00
359,Never Get Enough,20,Stacks of Soul Wax,"Marc Ferrari, Michael McGregor",2018-01-30 12:10:52+00:00
352,Eye Spy,16,Vol. 8: Spy Themes Remixed,"Chuck Cassey, Jules Bromley",2018-01-30 20:26:36+00:00
347,HIGH SPIRITS,147,Welcome to the Show,Christopher Franke,2018-01-31 04:30:37+00:00
346,Walk The Walk,16,ST172 From The Archives Vol. 9 - Retro Electronic,Taso Kotsos,2018-01-31 04:33:03+00:00
342,Filtered Club,10,Entertainment Fashion,"Alberto Dominguez, John Hunter, Jr., Jonathan ...",2018-01-31 05:00:58+00:00
339,The Journey Begins,4,Motivational - Volume 3,Larry Wolff,2018-01-31 08:31:01+00:00
338,Never Get Enough,9,Stacks of Soul Wax,"Marc Ferrari, Michael McGregor",2018-01-31 09:06:36+00:00
336,Eye Spy,16,Vol. 8: Spy Themes Remixed,"Chuck Cassey, Jules Bromley",2018-01-31 16:27:25+00:00
